In [34]:
import itertools

# Read input grid from a text file and remove newline characters
with open('/content/AoC_2023_Day14.txt') as f:
    txt = f.read()

grid = txt.replace('\n', '')

# Determine the grid dimensions
lines = txt.split('\n')
width = len(lines[0])
height = len(lines)

def reorder(sub, direction):
    """
    Reorder a substring of rocks based on the tilt direction.
    - 'n' or 'w' (north or west): Reversed sorted order.
    - 's' or 'e' (south or east): Sorted order.

    Args:
    sub (str): A substring of rocks and empty spaces.
    direction (str): The direction to reorder ('n', 's', 'w', 'e').

    Returns:
    str: Reordered substring.
    """
    return ''.join(reversed(sorted(sub)) if direction in 'nw' else sorted(sub))

def one_direction(grid, direction):
    """
    Tilt the grid in one direction, causing the rocks to move.

    Args:
    grid (str): A flattened representation of the grid.
    direction (str): The direction to tilt ('n', 's', 'w', 'e').

    Returns:
    str: The new grid state after tilting.
    """
    if direction in 'ns':
        # Tilt vertically (north or south)
        columns = [grid[i::height] for i in range(width)]  # Extract columns
        for i, col in enumerate(columns):
            # Reorder each column based on the tilt direction
            columns[i] = '#'.join(reorder(sub_col, direction) for sub_col in col.split('#'))
        return ''.join(itertools.chain(*zip(*columns)))  # Reconstruct the grid from columns
    else:  # 'we' for west or east
        # Tilt horizontally (west or east)
        rows = [grid[i * width:(i + 1) * width] for i in range(height)]  # Extract rows
        for i, row in enumerate(rows):
            # Reorder each row based on the tilt direction
            rows[i] = '#'.join(reorder(sub_row, direction) for sub_row in row.split('#'))
        return ''.join(rows)  # Reconstruct the grid from rows

def load(grid):
    """
    Calculate the total load on the north support beams after rocks have settled.

    Args:
    grid (str): A flattened representation of the grid.

    Returns:
    int: The total load on the north support beams.
    """
    columns = [grid[i::height] for i in range(width)]  # Extract columns
    # Calculate the load by summing up the load contribution of each rock in each column
    return sum(
        sum(row_index for row_index, cell in zip(range(len(col), 0, -1), col) if cell == 'O')
        for col in columns
    )

# Part 1: Calculate the load after tilting the grid north
north_tilted_grid = one_direction(grid, 'n')
load_north = load(north_tilted_grid)
print(f"Part 1: The total load on the north support beams after tilting north is {load_north}.")

# Part 2: Calculate the load after running the spin cycle for 1,000,000,000 cycles

# Initialize variables for cycle detection
cycle = 0
goal = 1_000_000_000

cycle_cache = {}  # Cache to store grid states and their corresponding cycle number
found_cycle = False  # Flag to indicate if a repeating cycle has been found

print("Part 2: Starting the spin cycle...")

while cycle < goal:
    # Perform a full spin cycle (north, west, south, east)
    for direction in 'nwse':
        grid = one_direction(grid, direction)

    cycle += 1  # Increment the cycle count

    # Check if the current grid state has been seen before
    if not found_cycle and (found_cycle := grid in cycle_cache):
        # If a cycle is found, calculate the cycle length
        cycle_length = cycle - cycle_cache[grid]
        # Fast forward by skipping unnecessary cycles
        skipped_cycles = cycle_length * ((goal - cycle) // cycle_length)
        cycle += skipped_cycles
        print(f"Cycle detected! Skipping {skipped_cycles} cycles...")
    else:
        # If no cycle is found, store the current grid state and cycle number
        cycle_cache[grid] = cycle

final_load = load(grid)
print(f"Part 2: After 1,000,000,000 cycles, the total load on the north support beams is {final_load}.")


Part 1: The total load on the north support beams after tilting north is 108759.
Part 2: Starting the spin cycle...
Cycle detected! Skipping 999999823 cycles...
Part 2: After 1,000,000,000 cycles, the total load on the north support beams is 89089.
